In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
os.getcwd()

# 5 tables in different levels

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Big_Lots_2019Q2_DCM_Frequency'

In [2]:
start_date=datetime.date(2019,5,5)
end_date=datetime.date(2019,6,23)


In [4]:
dcm_files_list=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Big_Lots_2019Q2_DCM_Frequency/BL_DCM_data/*.csv")

dcm_impr_list=sorted([x for x in dcm_files_list if "impr" in x])
dcm_click_list=sorted([x for x in dcm_files_list if "click" in x])


In [7]:
df_impr=pd.DataFrame()
df_click=pd.DataFrame()

for file in dcm_impr_list:
    df=pd.read_csv(file,dtype=str,usecols=['Event Time','User ID','Advertiser ID','Campaign ID','Site ID (DCM)','Placement ID','Impression ID'])
    df_impr=df_impr.append(df)
    
    
for file in dcm_click_list:
    df=pd.read_csv(file,dtype=str,usecols=['Event Time','User ID','Advertiser ID','Campaign ID','Site ID (DCM)','Placement ID','Impression ID'])
    df_click=df_click.append(df)




In [8]:
df_impr=df_impr.reset_index()
df_click=df_click.reset_index()

del df_impr['index']
del df_click['index']

df_impr['Event Time']=pd.to_datetime(df_impr['Event Time'],unit="us")
df_click['Event Time']=pd.to_datetime(df_click['Event Time'],unit="us")

df_impr['ETS_Time']=df_impr['Event Time'].apply(lambda x: x-datetime.timedelta(hours=5))
df_click['ETS_Time']=df_click['Event Time'].apply(lambda x: x-datetime.timedelta(hours=5))

df_impr['impression']=1
df_click['click']=1

In [9]:
df_impr=df_impr[df_impr['ETS_Time']>=start_date]
df_impr=df_impr[df_impr['ETS_Time']<=end_date]

df_click=df_click[df_click['ETS_Time']>=start_date]
df_click=df_click[df_click['ETS_Time']<=end_date]

df_impr=df_impr.reset_index()
del df_impr['index']
df_click=df_click.reset_index()
del df_click['index']

In [10]:
mapping_placement=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Big_Lots_2019Q2_DCM_Frequency/mapping_site_placement/placements_mapping_JL_2019-06-27.csv")
mapping_site=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Big_Lots_2019Q2_DCM_Frequency/mapping_site_placement/sites_mapping_JL_2019-06-27.csv")

mapping_site_only=mapping_site[['Site ID (DCM)','Site (DCM)']].drop_duplicates()
mapping_site_only=mapping_site_only.sort_values(["Site ID (DCM)","Site (DCM)"]).drop_duplicates("Site ID (DCM)")

In [11]:
df_impr_agg=df_impr.groupby(['User ID','Advertiser ID','Campaign ID','Site ID (DCM)','Placement ID'])['impression'].sum().to_frame().reset_index()
df_click_agg=df_click.groupby(['User ID','Advertiser ID','Campaign ID','Site ID (DCM)','Placement ID'])['click'].sum().to_frame().reset_index()


In [12]:
df_click_and_impr_agg=pd.merge(df_impr_agg,df_click_agg,on=['User ID','Advertiser ID','Campaign ID','Site ID (DCM)','Placement ID'],how="outer")
df_click_and_impr_agg['impression']=df_click_and_impr_agg['impression'].fillna(0)
df_click_and_impr_agg['click']=df_click_and_impr_agg['click'].fillna(0)
df_click_and_impr_agg['impr_and_click']=df_click_and_impr_agg['impression']+df_click_and_impr_agg['click']

In [13]:
df_id_site_impr_click=df_click_and_impr_agg.groupby(['User ID','Site ID (DCM)'])['impression','click','impr_and_click'].sum().reset_index()
df_id_site_impr_click.shape

(12264134, 5)

In [14]:
df_id_site_impr_click['Site ID (DCM)']=df_id_site_impr_click['Site ID (DCM)'].astype(str)
mapping_site_only['Site ID (DCM)']=mapping_site_only['Site ID (DCM)'].astype(str)
df_id_site_impr_click=pd.merge(df_id_site_impr_click,mapping_site_only,on="Site ID (DCM)",how="left")
df_id_site_impr_click.shape

(12264134, 6)

In [15]:
df_id_site_impr_click.head(2)

,User ID,Site ID (DCM),impression,click,impr_and_click,Site (DCM)
0,AMsySZY----Lex1O5Fk3ORX9cadI,5174983,1.0,0.0,1.0,Stack Adapt
1,AMsySZY---0BYUKWdkdTG_kvwMnP,5232462,1.0,0.0,1.0,MobileFuse


# Add layer of Rewards members

In [18]:
Google_LR_mapping=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Big_Lots_2019Q2_DCM_Frequency/mapping_GID_IDL/BL_merged_mapping_both_20190203_20190623_JL_2019-06-29.csv",dtype=str)
Google_LR_mapping=Google_LR_mapping.drop_duplicates()
print(Google_LR_mapping.shape)

LR_BL_mapping_2019Q1=pd.read_table('/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Big_Lots_2019Q2_DCM_Frequency/mapping_2019Q1_BL_LR/CL_BigLots_BL_mapping_file_2019Q1_JL_2019-05-14_20190516_111931_0000.psv.gz',
                                   dtype=str,sep="|",usecols=['Customer_Link','customer_id_hashed']).drop_duplicates()
print(LR_BL_mapping_2019Q1.shape)

11961511

In [47]:
userid_by_idlcount=Google_LR_mapping.groupby("User ID")['Customer_Link'].nunique().to_frame().reset_index().sort_values("Customer_Link",ascending=False)
userid_by_idlcount.head(5)

,User ID,Customer_Link
0,AMsySZY----pv7beTY8m8kTkF-1e,1
9970285,AMsySZadEpc3kOV3FdY-IcrfrU21,1
9970286,AMsySZadEpd_lsZX-FfDx53kiTO-,1
9970288,AMsySZadEphF6NueYns-awE1qlSZ,1
9970289,AMsySZadEpheQ3zHtPpCwXCoyJ4b,1


In [57]:
userid_by_idlcount[userid_by_idlcount['Customer_Link']!=1].shape

(486431, 3)

In [56]:
Google_LR_mapping[Google_LR_mapping['User ID']=="AMsySZa8BoiK3yN4C-vF0pgLHg6s"]

,Customer_Link,User ID
94338,XY1468xcJFLltNZhNOBNcifsl4Nv2FOinW2I74vOIK1xTOThs,AMsySZa8BoiK3yN4C-vF0pgLHg6s
8035848,XY1468BWmsaptl83iNRQeTgHofjCqSiJHS8vSRp0xKTYyFoKQ,AMsySZa8BoiK3yN4C-vF0pgLHg6s
8284200,XY1468WZ8scGZTSbEaKjN9RhhTiE0ZvdiQGB7KG8vMCql2zms,AMsySZa8BoiK3yN4C-vF0pgLHg6s
9793743,XY146816b6hxgey2dnOEmaMaQmiVaE5GfcfPxK4B7Hu7DCR4s,AMsySZa8BoiK3yN4C-vF0pgLHg6s
11354100,XY1468evq1DjN1QxNvMhOl7HAY463pu9NmbO7G4VBwo79hvzs,AMsySZa8BoiK3yN4C-vF0pgLHg6s
11772375,XY1468jVU81UDqMKN9snCm0zvtKG5MMlyWHIFXJJwM3iJYj_Q,AMsySZa8BoiK3yN4C-vF0pgLHg6s
12665646,XY1468f5TuPnJjrNE3uNYf03aD6Vh8_IitaKY3f7Nw5J0SWLE,AMsySZa8BoiK3yN4C-vF0pgLHg6s
13105221,XY1468K-1jVivXxkFiTw9o32EWri9kKvUJjGscR-QcOBDyBCE,AMsySZa8BoiK3yN4C-vF0pgLHg6s
13257366,XY1468GaTQubOutMe5_9C9_pt_pCTMqw7D6k02TKNbWg9Aamk,AMsySZa8BoiK3yN4C-vF0pgLHg6s
13473247,XY1468B1juQjnyH8zPqIVMPENb4BJD8T_kH_a05fXc1aU1bP4,AMsySZa8BoiK3yN4C-vF0pgLHg6s


In [37]:
Google_LR_mapping.shape

(15570022, 2)

In [38]:
Google_LR_mapping.head(2)

,Customer_Link,User ID
0,XY1468H7T0H8Fs_SvVpVceBEUpqSNmFgpKXFRLOYBLp9ooDJI,AMsySZZXZEhBO4_kLZyEsNpnXzDC
1,XY1468iw-6onyz7TTQ30QLPvDYuN9rvYN5FtYbcpEZYsF4vpY,AMsySZZZIURPM1l9umy3cVaRUDnR


In [39]:
Google_LR_mapping['User ID'].nunique()

15078674

In [49]:
Google_LR_mapping['Customer_Link'].nunique()

12119057

In [43]:
LR_BL_mapping_2019Q1.head(2)

,Customer_Link,customer_id_hashed
0,Xi1468nfC3a_gGXZVi7-7naeMJQiaRuoQjh1l0Z7f2pekG...,57e0561be9e66bb2e3d834ae1c87b51ff460d498446d69...
1,XY1468rh0PXac0u_86Vh2QhqH1Z0BkT-CJlnfc9Hp69rbJGhM,5a692ab06280b862e40870d622abbf64c3ec663efc1f8a...


In [42]:
LR_BL_mapping_2019Q1['customer_id_hashed'].nunique()

20753480

In [44]:
LR_BL_mapping_2019Q1['Customer_Link'].nunique()

20514334

In [45]:
LR_BL_mapping_2019Q1.shape

(20765842, 2)

In [46]:
LR_BL_mapping_2019Q1.drop_duplicates().shape

(20765842, 2)

In [36]:
df_id_site_impr_click=pd.merge(df_id_site_impr_click,)

(15570022, 2)
(20765842, 2)


# Add layer of Rewards Shoppers

In [16]:
# Sales Trans ID counts

(15570022, 2)

In [35]:
Google_LR_mapping.head(2)

,Customer_Link,User ID
0,XY1468H7T0H8Fs_SvVpVceBEUpqSNmFgpKXFRLOYBLp9ooDJI,AMsySZZXZEhBO4_kLZyEsNpnXzDC
1,XY1468iw-6onyz7TTQ30QLPvDYuN9rvYN5FtYbcpEZYsF4vpY,AMsySZZZIURPM1l9umy3cVaRUDnR


# Add layer of Rewards Segments

In [17]:
# Q2 regular segments
# No new sign ups because of the unavailibility of the mapping from BL id to Google User ID

In [12]:
df_id_site_impr_click=df_id_site_impr_click.fillna(0)

site_impr_only=[]
site_click_only=[]
site_both=[]
for site, group in df_id_site_impr_click.groupby(['Site (DCM)']):
    if group['click'].sum()==0:
        site_impr_only=site_impr_only+[site]
    elif group['impression'].sum()==0:
        site_click_only=site_click_only+[site]
    else:
        site_both=site_both+[site]
    
df_click_only=pd.DataFrame({"site_type":["click_only"]*len(site_click_only),"Site (DCM)":site_click_only})
df_impr_only=pd.DataFrame({"site_type":["impr_only"]*len(site_impr_only),"Site (DCM)":site_impr_only})
df_both=pd.DataFrame({"site_type":["impr_and_click"]*len(site_both),"Site (DCM)":site_both})

df_all_site=df_click_only.append(df_impr_only).append(df_both)
df_id_site_impr_click=pd.merge(df_id_site_impr_click,df_all_site,on="Site (DCM)",how="left")

In [13]:
df_id_site_impr_click.head(2)

,User ID,Site ID (DCM),impression,click,impr_and_click,Site (DCM),site_type
0,AMsySZY----Lex1O5Fk3ORX9cadI,5174983,1.0,0.0,1.0,Stack Adapt,impr_and_click
1,AMsySZY---0BYUKWdkdTG_kvwMnP,5232462,1.0,0.0,1.0,MobileFuse,impr_and_click


In [15]:
rewards_mapping_2019Q1=pd.read_csv("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Big_Lots_2019Q2_DCM_Frequency/CL_BigLots_BL_mapping_file_2019Q1_JL_2019-05-14_20190516_111931_0000.psv.gz",
                                  sep="|",compression="gzip",dtype=str,usecols=['Customer_Link','customer_id_hashed'])
print(rewards_mapping_2019Q1.shape)

rewards_mapping_2019Q1.head(2)

(20767278, 2)


,Customer_Link,customer_id_hashed
0,Xi1468nfC3a_gGXZVi7-7naeMJQiaRuoQjh1l0Z7f2pekG...,57e0561be9e66bb2e3d834ae1c87b51ff460d498446d69...
1,XY1468rh0PXac0u_86Vh2QhqH1Z0BkT-CJlnfc9Hp69rbJGhM,5a692ab06280b862e40870d622abbf64c3ec663efc1f8a...


In [18]:
all_files_2019Q2=glob.glob("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q2/output_final_20190422/Test_and_Control/uploaded/*.csv")
print("len(all_files_2019Q2): "+str(len(all_files_2019Q2)))

df_all_2019Q2_segments=pd.DataFrame()

for file in all_files_2019Q2:
    df=pd.read_csv(file,dtype=str)
    df_all_2019Q2_segments=df_all_2019Q2_segments.append(df)
print(df_all_2019Q2_segments.shape)
df_all_2019Q2_segments.head(3)

(21007241, 3)


,customer_id_hashed,email_address_hash,segment
0,51ce14856c1e1eae5156bc18e4bd2a63d8412462aab055...,80318c3f741c9a0cc18b19a500113b3e0dff9b9f5cc7e0...,T_Quadrant III_P_H_2019Q2_RFM
1,580dd10a059dadb652bf9da7391ef49590d1019428dc2e...,8c3335c9ec1496e33f13f1222e6764b621afa554feb1a5...,T_Quadrant III_P_H_2019Q2_RFM
2,9fe7543c74aadb2f8c2889d45ab9b406489a9436c098c1...,82f9a05e4f75783d5079397966bdca9cf71d83329d71e1...,T_Quadrant III_P_H_2019Q2_RFM


In [31]:
df_all_2019Q2_segments.head(4)

,customer_id_hashed,segment,T_C_Group
0,51ce14856c1e1eae5156bc18e4bd2a63d8412462aab055...,T_Quadrant III_P_H_2019Q2_RFM,T
1,580dd10a059dadb652bf9da7391ef49590d1019428dc2e...,T_Quadrant III_P_H_2019Q2_RFM,T
2,9fe7543c74aadb2f8c2889d45ab9b406489a9436c098c1...,T_Quadrant III_P_H_2019Q2_RFM,T
3,c5f55af0dce7ff2dbae6da4f74af7bec3e1a7a9a50e381...,T_Quadrant III_P_H_2019Q2_RFM,T


In [19]:
df_all_2019Q2_segments['T_C_Group']=df_all_2019Q2_segments['segment'].apply(lambda x: x.split("_")[0])
print(df_all_2019Q2_segments['T_C_Group'].unique())

del df_all_2019Q2_segments['email_address_hash']
gc.collect()

array(['T', 'C'], dtype=object)

In [27]:
df_all_2019Q2_segments.head(2)

,customer_id_hashed,segment,T_C_Group
0,51ce14856c1e1eae5156bc18e4bd2a63d8412462aab055...,T_Quadrant III_P_H_2019Q2_RFM,T
1,580dd10a059dadb652bf9da7391ef49590d1019428dc2e...,T_Quadrant III_P_H_2019Q2_RFM,T


In [21]:
rewards_mapping_2019Q1.head(2)

,Customer_Link,customer_id_hashed
0,Xi1468nfC3a_gGXZVi7-7naeMJQiaRuoQjh1l0Z7f2pekG...,57e0561be9e66bb2e3d834ae1c87b51ff460d498446d69...
1,XY1468rh0PXac0u_86Vh2QhqH1Z0BkT-CJlnfc9Hp69rbJGhM,5a692ab06280b862e40870d622abbf64c3ec663efc1f8a...


In [32]:
rewards_mapping_2019Q1.shape

(20767278, 2)

In [26]:
rewards_mapping_2019Q1['customer_id_hashed'].nunique()

20753480

In [ ]:
output_1_overall_by_id_group=df_id_site_impr_click.groupby(['User ID'])['impression','click','impr_and_click'].sum().reset_index()
output_1_overall_by_id_group['id_expose_group']=np.where(((output_1_overall_by_id_group['impression']>0) & (output_1_overall_by_id_group['click']>0)),"Impr_and_Click",
                                               np.where(((output_1_overall_by_id_group['impression']>0) & (output_1_overall_by_id_group['click']==0)),"Impr_only",
                                                        np.where(((output_1_overall_by_id_group['impression']==0) & (output_1_overall_by_id_group['click']>0)),"Click_only","Others")
                                                       )
                                               )

In [ ]:
def quantile_25(x):
    return np.percentile(x,25)
def quantile_50(x):
    return np.percentile(x,50)
def quantile_75(x):
    return np.percentile(x,75)

func_group_output={"User ID":"nunique",
                     'impression':['sum','mean','min',quantile_25,quantile_50,quantile_75,'max'], 
                     'click':['sum','mean','min',quantile_25,quantile_50,quantile_75,'max'],
                     'impr_and_click':['sum','mean','min',quantile_25,quantile_50,quantile_75,'max']}


In [15]:
# Overall idgroup and then each site

mapping_id_group=output_1_overall_by_id_group[['User ID','id_expose_group']]

output_0_overall_by_id_group_site=df_id_site_impr_click.copy()
output_0_overall_by_id_group_site=pd.merge(output_0_overall_by_id_group_site,mapping_id_group,on="User ID",how="left")
output_0_overall_by_id_group_site=output_0_overall_by_id_group_site.groupby(['id_expose_group','Site (DCM)'])['impression','click','impr_and_click'].agg(func_group_output).reset_index()

new_cols=[]
for col in output_0_overall_by_id_group_site.columns.tolist():
    if col[1]=="":
        col_new=col[0]
    else:
        col_new=col[0]+"_"+col[1]
    new_cols=new_cols+[col_new]
    
output_0_overall_by_id_group_site.columns=new_cols


In [16]:
# Each site and then the group of each site

output_by_site_then_group=pd.DataFrame()

for site,group in df_id_site_impr_click.groupby('Site (DCM)'):
    group['id_group_of_site']=np.where(((group['impression']>0) & (group['click']>0)),"site_Impr_and_Click",
                                               np.where(((group['impression']>0) & (group['click']==0)),"site_Impr_only",
                                                        np.where(((group['impression']==0) & (group['click']>0)),"site_Click_only","Others")
                                                       )
                                               )
    group=group.groupby(['Site (DCM)','id_group_of_site'])['impression','click','impr_and_click'].agg(func_group_output).reset_index()
    
    new_cols=[]
    for col in group.columns.tolist():
        if col[1]=="":
            col_new=col[0]
        else:
            col_new=col[0]+"_"+col[1]
        new_cols=new_cols+[col_new]
    group.columns=new_cols
    
    output_by_site_then_group=output_by_site_then_group.append(group)


    


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [17]:
output_1_overall_by_id_group=output_1_overall_by_id_group.groupby(['id_expose_group'])['impression','click','impr_and_click'].agg(func_group_output).reset_index()

new_cols=[]
for col in output_1_overall_by_id_group.columns.tolist():
    if col[1]=="":
        col_new=col[0]
    else:
        col_new=col[0]+"_"+col[1]
    new_cols=new_cols+[col_new]
    
output_1_overall_by_id_group.columns=new_cols


In [18]:
output_2_overall_by_site_group=df_id_site_impr_click.groupby(['site_type'])['impression','click','impr_and_click'].agg(func_group_output).reset_index()

new_cols=[]
for col in output_2_overall_by_site_group.columns.tolist():
    if col[1]=="":
        col_new=col[0]
    else:
        col_new=col[0]+"_"+col[1]
    new_cols=new_cols+[col_new]
    
output_2_overall_by_site_group.columns=new_cols

In [19]:
output_3_overall_by_site=df_id_site_impr_click.groupby(['Site (DCM)'])['impression','click','impr_and_click'].agg(func_group_output).reset_index()

new_cols=[]
for col in output_3_overall_by_site.columns.tolist():
    if col[1]=="":
        col_new=col[0]
    else:
        col_new=col[0]+"_"+col[1]
    new_cols=new_cols+[col_new]
    
output_3_overall_by_site.columns=new_cols

In [20]:
output_3_overall_by_site=pd.merge(output_3_overall_by_site,df_all_site,on="Site (DCM)",how="left")
output_3_overall_by_site=output_3_overall_by_site[['Site (DCM)','site_type']+[x for x in output_3_overall_by_site.columns.tolist() if (x not in ['Site (DCM)','site_type'])]]


In [21]:
writer_folder='/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Big_Lots_2019Q2_DCM_Frequency/output_'+str(datetime.datetime.now().date())+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

In [22]:
# 5 tables in different levels

writer=pd.ExcelWriter(writer_folder+"BL_2019Q2_"+str(start_date)+"_"+str(end_date)+"_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_1_overall_by_id_group.to_excel(writer,"by_id_group",index=False)
output_0_overall_by_id_group_site.to_excel(writer,"by_id_group_site",index=False)
output_by_site_then_group.to_excel(writer,"by_site_id_group",index=False)
output_2_overall_by_site_group.to_excel(writer,"by_site_group",index=False)
output_3_overall_by_site.to_excel(writer,"by_site",index=False)
writer.save()